In [3]:
# imdb

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pickle
import csv
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score

dataset = 'imdb' # folder and dataset name
# cols = ['gender', 'age', 'politics'] # outcome columns
cols = ['label_x'] # outcome columns
outcome = 'label'

num_epochs = 500
lr = 0.003

df = pd.read_csv("data/" + dataset + ".csv")

print(dataset)

# for model in ['GensimLDA', 'BERTopic', 'NMF', 'Mallet_LDA', 'CTM']:
for model in ['BERTopic']:
    print(model)
    scores = []
    scores_logit = []
    scores_rf = []
    scores_old = []

    rmses = []
    r2s = []
    maes = []


    for run in range(2, 3):

        print("Run: " + str(run))

        # loading in distributions that were saved during topic extraction
        test_distribution = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_topic_distribution_test.pkl', 'rb'))
        train_distribution = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_topic_distribution_train.pkl', 'rb'))
        train = pickle.load(open(dataset + '/BERTopic/run_' + str(run) + '/train.pkl', 'rb'))
        test = pickle.load(open(dataset + '/BERTopic/run_' + str(run) + '/test.pkl', 'rb'))

        topics = []
        with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_100.txt', 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                topic_list = [item.strip() for item in row if item.strip()]
                topics.append(topic_list)

        temp = pd.concat([train, test]).reset_index(drop=True) # concatenating train and test datasets
        distribution = np.concatenate([train_distribution, test_distribution]) # concatenating train and test distributions

        merged = pd.merge(temp, df, how='inner', left_on = 'message_id', right_on = 'Unnamed: 0')[['message_id', 'message_x'] + cols]
        merged.columns = ['message_id', 'message', 'label']

        X = distribution
        y = merged[outcome].reset_index(drop=True) # the outcome we care about

        # # 80-20 split --> didn't use train-test-split function since its already shuffled
        X_train = X[:round(0.80 * len(X))]
        X_test = X[round(0.80 * len(X)):]

        y_train = y[:round(0.80 * len(X))]
        y_test = y[round(0.80 * len(X)):]

        # Convert arrays to torch tensors
        X_train_tensor = torch.tensor(np.array(X_train).astype(np.float32))
        y_train_tensor = torch.tensor(np.array(y_train).astype(np.longlong))  # Use long for classification
        X_test_tensor = torch.tensor(np.array(X_test).astype(np.float32))
        y_test_tensor = torch.tensor(np.array(y_test).astype(np.longlong))

        # Create datasets and dataloaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

        train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

        # Define the model
        class LogisticRegressionModel(nn.Module):
            def __init__(self, input_size, num_classes):
                super(LogisticRegressionModel, self).__init__()
                self.layer1 = nn.Linear(input_size, 300)  # First linear layer

            def forward(self, x):
                return self.layer1(x)

        input_size = X_train.shape[1]
        num_classes = len(np.unique(y_train))  # Assuming y_train contains all classes
        model = LogisticRegressionModel(input_size, num_classes)

        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()  # This includes softmax
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Train the model
        for epoch in range(num_epochs):
            for inputs, targets in train_loader:
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            if (epoch+1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # Predict on the test set
        model.eval()  # Set the model to evaluation mode

        with torch.no_grad():
            y_pred_train = model(X_train_tensor)
            _, predicted_train = torch.max(y_pred_train.data, 1)

        # Calculate accuracy
        score_logit_train = accuracy_score(predicted_train.numpy(), y_train)
        print(f'Train Accuracy Score: {score_logit_train}')

        with torch.no_grad():
            y_pred_test = model(X_test_tensor)
            _, predicted = torch.max(y_pred_test.data, 1)

        # Calculate accuracy
        score_logit = accuracy_score(predicted.numpy(), y_test)
        scores_logit.append(score_logit)
        print(f'Test Accuracy Score: {score_logit}')
    print(np.mean(scores_logit))
    print()

imdb
BERTopic
Run: 2
Epoch [10/500], Loss: 0.6990
Epoch [20/500], Loss: 0.6917
Epoch [30/500], Loss: 0.6889
Epoch [40/500], Loss: 0.6863
Epoch [50/500], Loss: 0.6836
Epoch [60/500], Loss: 0.6809
Epoch [70/500], Loss: 0.6783
Epoch [80/500], Loss: 0.6757
Epoch [90/500], Loss: 0.6732
Epoch [100/500], Loss: 0.6707
Epoch [110/500], Loss: 0.6683
Epoch [120/500], Loss: 0.6660
Epoch [130/500], Loss: 0.6637
Epoch [140/500], Loss: 0.6615
Epoch [150/500], Loss: 0.6594
Epoch [160/500], Loss: 0.6573
Epoch [170/500], Loss: 0.6552
Epoch [180/500], Loss: 0.6532
Epoch [190/500], Loss: 0.6513
Epoch [200/500], Loss: 0.6493
Epoch [210/500], Loss: 0.6474
Epoch [220/500], Loss: 0.6456
Epoch [230/500], Loss: 0.6438
Epoch [240/500], Loss: 0.6420
Epoch [250/500], Loss: 0.6402
Epoch [260/500], Loss: 0.6384
Epoch [270/500], Loss: 0.6367
Epoch [280/500], Loss: 0.6350
Epoch [290/500], Loss: 0.6334
Epoch [300/500], Loss: 0.6317
Epoch [310/500], Loss: 0.6301
Epoch [320/500], Loss: 0.6285
Epoch [330/500], Loss: 0.626